In [5]:
import numpy as np
import random
import math

In [2]:
def avg_var_cal(R_T, R_E, Es_dBm, Noise_Var_E, Noise_Var_B, rho_E, rho_B):
    R_S = R_T - R_E
    Es = 10**(Es_dBm/10)/1000
    sigma_E = math.sqrt(Noise_Var_E)
    sigma_B = math.sqrt(Noise_Var_B)
    avg_SNR_B = (2*Es) / (sigma_B)^2
    avg_SNR_E = (2*Es) / (sigma_E)^2
    Noise_Var_E_v = 1 - rho_E**2
    Noise_Var_B_v = 1 - rho_B**2
    return avg_SNR_B, avg_SNR_E, Noise_Var_B_v, Noise_Var_E_v

In [2]:
def TS_cha_ge(iteration, rho_B, rho_E, Noise_Var_B_v):
    for k in range(iteration):
        H_B1 = (np.random.randn(2,1) + math.sqrt(-1)*np.random.randn(2,1) / math.sqrt(2))
        H_B2 = np.multiply(rho_B, H_B1) + np.divide(np.multiply((np.random.randn(2,1) + math.sqrt(-1)*np.random.randn(2,1)),
                math.sqrt(Noise_Var_B_v)), math.sqrt(2))
        H_B3 = H_B1.extend(H_B2)
        H_B3_JML = H_B1.extend(H_B1)
        H_B = [ [H_B3_JML(1,1), H_B3_JML(2,1)],
                [np.conj(H_B3_JML(2,2)), -np.conj(H_B3_JML(1,2))] ]

In [69]:
rho_B = 0.95
Noise_Var_B = 1
Noise_Var_B_v = 1 - rho_B**2

H_B1 = np.divide(np.random.randn(2,1) + np.multiply(1j,np.random.randn(2,1)), math.sqrt(2))
H_B2 = np.multiply(rho_B, H_B1) + np.divide(np.multiply((np.random.randn(2,1) + 1j*np.random.randn(2,1)),
        math.sqrt(Noise_Var_B_v)), math.sqrt(2))
H_B3 = np.c_[H_B1, H_B2]
H_B3_JML = np.c_[H_B1, H_B1]
# H_B = np.append([H_B3_JML[1,1], H_B3_JML[2,1]],
#          [np.conj(H_B3_JML[2,2]), -np.conj(H_B3_JML[1,2])])
print(H_B3_JML[0,1])

(1.0524607866199698-0.38028782311399206j)
